<a href="https://colab.research.google.com/github/mulcahrj/DATA6545_Final/blob/main/Random_Forest_Randomized_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
path = 'https://raw.githubusercontent.com/mulcahrj/DATA6545_Final/main/selected_features_train.csv'
features_train = pd.read_csv(path)

path = 'https://raw.githubusercontent.com/mulcahrj/DATA6545_Final/main/selected_features_test.csv'
features_test = pd.read_csv(path)

In [ ]:
frames = [features_test, features_train]

data = pd.concat(frames)

In [ ]:
#Split dataset into inputs and outputs
X = data.values[:, :-1] 
y = data.values[:, -1] 

In [ ]:
#Initiate the LR model with random hyperparameters
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')
rf = RandomForestClassifier(n_estimators=100, random_state=2019)

In [ ]:
rf.fit(X,y)

RandomForestClassifier(random_state=2019)

In [ ]:
rf.score(X,y)

0.9999779283554417

In [ ]:
# You will need the following dependencies for applying Cross-validation and evaluating the cross-validated score

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

# Build the k-fold cross-validator

kfold = KFold(n_splits=3, random_state=2019, shuffle=True)

In [ ]:
result = cross_val_score(rf, X, y, cv=kfold, scoring='accuracy')
print(result.mean())

0.8930187158261019


In [ ]:
n_estimators = [200,300,400]
max_features = ['auto', 'sqrt', 'log2']
max_depth = [4,6,8]
criterion = ['gini', 'entropy']
param_grid = dict(n_estimators=n_estimators, max_features=max_features, max_depth=max_depth, criterion = criterion)

In [ ]:
import numpy as np
import random
np.random.seed(7)
n_estimators1 = np.random.randint(200,300,400)
max_features1 = ['auto', 'sqrt', 'log2']
max_depth1 = np.random.randint(4,6,8)
criterion1 = ['gini', 'entropy']
param_grid1 = dict(n_estimators=n_estimators1, max_features=max_features1, max_depth=max_depth1, criterion = criterion1)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
random = RandomizedSearchCV(estimator=rf, param_distributions=param_grid1, cv = 3, n_jobs=-1)

import time
start_time = time.time()
random_result = random.fit(X, y)
# Summarize results
print("Best: %f using %s" % (random_result.best_score_, random_result.best_params_))
print("Execution time: " + str((time.time() - start_time)) + ' ms')

Best: 0.898625 using {'n_estimators': 219, 'max_features': 'auto', 'max_depth': 4, 'criterion': 'entropy'}
Execution time: 131.15918469429016 ms


##Evaluation

In [ ]:
# import required package for data handling
import pandas as pd
import numpy as np

# import required packages for splitting data
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

# import required packages for evaluating models
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support

# balance the data
from imblearn.over_sampling import SMOTE

In [ ]:
# define features and target
X = data.iloc[:,:-1].values
y = data.iloc[:,-1].values

In [ ]:
# Resampling and Balancing the data
sm = SMOTE(random_state = 2021) 
X_res, Y_res = sm.fit_resample(X, y) 

In [ ]:
clf = RandomForestClassifier(criterion= 'entropy', max_depth= 4, max_features= 'auto', n_estimators= 219)

In [ ]:
def my_eval(X, y, classifer = clf, k=10, scoring = 'f1'):
  '''
  return evaluation results (f1-score or ROC_AUC). 
  Built in k-fold evaluation.
  INPUTS:
  ----
  - X: features; DataFrame or Numpy ndarray;
  - y: target; DataFrame or Numpy ndarray;
  - classifier: any sklearn (or its add-on) based classifier
  - k: number of folds in cross validation
  - scoring: evaluation metric ('f1' default or 'roc_auc')
  OUTPUT:
  ----
  bias/variance score of selected metric.
  '''
  scores = []
  for i in range(50):
    #### generate random numbers to shuffle the data for training and test
    np.random.seed(2021)
    random_int = np.random.randint(0,3000)
    #### create cross validation folds
    kfold = model_selection.KFold(n_splits=k, random_state=random_int, shuffle=True)
    #### record the score
    score = model_selection.cross_val_score(clf, X=X, y=y, cv=kfold, scoring=scoring)
    scores.append(score)
  scores = np.array(scores)
  #### we need to calculate the bias (average score) and viariance (std)
  bias, variance = round(scores.mean(),4), round(scores.std(),4)
  return(bias, variance)

In [ ]:
# getting averaged f1_score from 10-fold CV (default)
my_eval(X_res, Y_res, clf, 10)

In [ ]:
# getting averaged ROC_AUC from 10-fold CV
my_eval(X_res, Y_res, clf, 10, 'roc_auc')